In [2]:
import pandas as pd
import matplotlib.pyplot as plt
# Modeling
from neuralforecast import NeuralForecast
from neuralforecast.models import LSTM #DeepAR,NHITS,RNN,

import torch
from typing import Union
from neuralforecast.losses.pytorch import MSE
from neuralforecast.losses.pytorch import BasePointLoss
from neuralforecast import NeuralForecast
from neuralforecast.models import LSTM

from neuralforecast.utils import augment_calendar_df



from utils.losses import customLoss

In [ ]:
df = pd.read_csv('data/01_input_history.csv')
df.head()

In [ ]:
# pre process
from utils import preprocess

df_train_null, df_train_inactive, df_train_active, df_validation = preprocess.preprocess_ex1(df)

In [ ]:
# merge the inactive to the active
# print(df_active.shape)
# print(df_inactive.shape)

df_train_merged = pd.merge(df_train_active, df_train_inactive, how='outer', on=['unique_id', 'ds', 'Quantity', 'Country', 'Product'])
# df_merged.head()


# create a static dataframe
df_train_static = df_train_merged[['unique_id', 'Country', 'Product']].drop_duplicates().reset_index(drop=True)
df_train_static = pd.get_dummies(df_train_static, columns=['Country', 'Product'], drop_first=True)
assert df_train_static.shape[0] == df_train_merged['unique_id'].nunique(), 'The number of unique_id in static and merged dataframes do not match!'
# df_static.head()

In [ ]:
df_train_merged = augment_calendar_df(df_train_merged, freq='M')[0]
df_validation = augment_calendar_df(df_validation, freq='M')[0]

In [ ]:
FORECASTING_HORIZON = 12 # one year ahead forecast

In [ ]:
custom_loss = customLoss()

nf = NeuralForecast(
    models=[
        # Model 1: Long-short term memory
        LSTM(
            h =FORECASTING_HORIZON,
            input_size= FORECASTING_HORIZON*2,
            loss=custom_loss,
            encoder_n_layers = 2,
            encoder_hidden_size= 128,
            decoder_hidden_size=  128,
            decoder_layers= 2,
            futr_exog_list = ['month'],
            stat_exog_list = df_train_static.columns.tolist()[1:],
            batch_size = 128,
            learning_rate= 1e-3,
            max_steps = 2000,
            scaler_type = None,
            random_seed=42
        )
        # Model 2: 
        
    ],
    freq='MS',

)

In [ ]:
nf.fit(
    df = df_train_merged[['unique_id','ds','Quantity',  'month']],
    static_df= df_train_static,
    #val_size= 12
    # id_col = 'unique_id'
    target_col = 'Quantity'
)

In [1]:
# save the model 
nf.save('models/lstm_model')

NameError: name 'nf' is not defined

In [ ]:
# useful_ids = df_train_merged['unique_id'].unique()
# df_validation_raw = df_validation[df_validation['unique_id'].isin(useful_ids)]

In [ ]:
# future_dataframe = nf.make_future_dataframe()
# future_dataframe.shape
# df_validation_raw['ds'] =df_validation['ds'] + pd.offsets.MonthEnd(0)